### We are now using Poolynomial Regression to fit non-linear data and the model will not be just a line  
### Smart to use with another linear model, as this model is heavily affected by outliers

In [14]:
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

In [4]:
os.chdir("C:/Users/ab/Documents/GitHub/data-science-lectures-monte/data")

In [5]:
df = pd.read_csv("titanic_train.csv")

In [6]:
print(df.head())

   passenger_id  pclass                                               name  \
0          1216       3                                 Smyth, Miss. Julia   
1           699       3                                    Cacic, Mr. Luka   
2          1267       3  Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...   
3           449       2              Hocking, Mrs. Elizabeth (Eliza Needs)   
4           576       2                                    Veal, Mr. James   

      sex   age  sibsp  parch  ticket     fare cabin embarked boat  body  \
0  female   NaN      0      0  335432   7.7333   NaN        Q   13   NaN   
1    male  38.0      0      0  315089   8.6625   NaN        S  NaN   NaN   
2  female  30.0      1      1  345773  24.1500   NaN        S  NaN   NaN   
3  female  54.0      1      3   29105  23.0000   NaN        S    4   NaN   
4    male  40.0      0      0   28221  13.0000   NaN        S  NaN   NaN   

                  home.dest  survived  
0                       NaN       

In [23]:
# clean up the data, as usual

columns = ["sex", "age", "pclass", "sibsp"]

df.loc[df["sex"]=="male", "sex"] = 0
df.loc[df["sex"]=="female", "sex"] = 1

df["age"] = df["age"].fillna(df["age"].median())

In [25]:
# just a reminder, we are making the 4 columns into the independent variables, and also making sex binary while cleaning age
# then we can select the 4 columns into x, and the 1 into y

x = df[columns]
y = df["survived"]

In [26]:
# create the NEW polynomial features, kinda confusing but basiclaly this adds new "simulated" features in order to create a quadratic equation
# basically making the model more curvy

polynomial = preprocessing.PolynomialFeatures(degree=4, interaction_only=True)
polyFeatures = polynomial.fit_transform(x)

In [27]:
#  create the *additional* regression to assist the polynomial regression, this time its logistic regression

reg = LogisticRegression(max_iter=2000)
reg.fit(polyFeatures, y)
print(reg.score(polyFeatures, y))

0.8164705882352942


In [28]:
# our accuracy ends up much better now, lets see it vs the test data
# same steps as for the training data

test = pd.read_csv("titanic_test.csv")

test.loc[test["sex"]=="male", "sex"] = 0
test.loc[test["sex"]=="female", "sex"] = 1

test["age"] = test["age"].fillna(test["age"].median())

In [29]:
x_test = test[columns]

In [30]:
x_test = polynomial.fit_transform(x_test)

In [31]:
pred = reg.predict(x_test)

In [32]:
print(pred)

[0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 1 1 0 0
 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0
 0 1 0 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1
 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1
 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0
 0 1 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1
 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1]


In [33]:
# zip the data into a dataframe

dataset = pd.DataFrame(list(zip(test["passenger_id"].to_list(), pred.tolist())), columns=["passenger_id", "survived"])

In [34]:
print(dataset.head())

   passenger_id  survived
0           295         0
1          1150         0
2            89         0
3          1063         0
4          1020         0


In [36]:
# create the submission csv

os.chdir("C:/Users/ab/Documents/GitHub/data-science-lectures-monte/submissions")
dataset.to_csv("submission_polynomial_regression.csv", index=False)

## Our result shows us that some of our data is "nonlinear", and that using polynomial regression is more accurate for this dataset